In [1]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 292 kB 19.7 MB/s eta 0:00:01


In [2]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 110 kB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 287 kB 29.9 MB/s eta 0:00:01


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 1.8 MB 19.2 MB/s eta 0:00:01     |██████████████████▎             | 1.0 MB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 22.0 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=9266ed6a1ff8a0972b481f04a5f043e7a5fa0eeb44294d5bf85dd548d8155639
  Stored in directory: /tmp/pip-ephem-wheel-cache-bz5_iisp/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=c73a31428b871da5b689bed9b7049cafdf08c432dd52bd64305a53a2fd375add
  Stored in directory: /tmp/pip-ephem-wheel-cache-bz5_iisp/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built promise p

In [5]:
import wandb

In [6]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import wandb
run = wandb.init()
artifact = run.use_artifact('alokpadhi/Sentiment Analysis-Hinglish/run-3rxv0ijv-ProcessTrainDataframe:v0', type='run_table')
artifact_dir = artifact.download()

wandb: Currently logged in as: alokpadhi. Use `wandb login --relogin` to force relogin


In [9]:
artifact = run.use_artifact('alokpadhi/Sentiment Analysis-Hinglish/run-3rxv0ijv-ProcessTestDataframe:v0', type='run_table')
artifact_dir = artifact.download()

In [10]:
import json

In [16]:
import pandas as pd

In [20]:
with open("artifacts/run-3rxv0ijv-ProcessTrainDataframe:v0/Process Train Dataframe.table.json", 'r') as fp:
    train_data = json.load(fp)

In [23]:
train_df = pd.DataFrame(train_data["data"], columns=train_data["columns"])
train_df.head()

,Sentence,Sentiment,Sentiment Label
0,nen á vist bolest vztek smutek zmatek osam ě l...,neutral,2
1,Haan yaar neha 😔😔 kab karega woh post 😭 Usne n...,neutral,2
2,television media congress ke liye nhi h . Ye t...,negative,1
3,All India me nrc lagu kare w Kashmir se dhara ...,positive,0
4,Pagal hai kya ? They aren ’ t real issues Mand...,neutral,2


In [24]:
with open("artifacts/run-3rxv0ijv-ProcessTestDataframe:v0/Process Test Dataframe.table.json", 'r') as fp:
    test_data = json.load(fp)

In [25]:
test_df = pd.DataFrame(test_data["data"], columns=test_data["columns"])
test_df.head()

,Sentence,Sentiment,Sentiment Label
0,modi mantrimandal may samil honay par badhai n...,positive,0
1,Tu toh naamakool hai Mare h,negative,1
2,YOU saw caste and religion in them ... nation ...,negative,1
3,sir local police station pe complaint krne par...,neutral,2
4,Ve Maahi song from # Kesari is current favouri...,positive,0


In [26]:
import re
import time
import string
import contractions
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import metrics

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [28]:
train_df.shape

(14000, 3)

In [29]:
test_df.shape

(3000, 3)

In [30]:
X = train_df.Sentence.to_numpy()
y = train_df["Sentiment Label"].to_numpy()

In [31]:
X_test = test_df.Sentence.to_numpy()
y_test = test_df["Sentiment Label"].to_numpy()

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [33]:
len(X_train), len(X_val), len(X_test)

(11200, 2800, 3000)

In [51]:
class Vocab(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]
    
    def __len__(self):
        return len(self.word2idx)

In [52]:
def build_vocab(sentences, threshold=15):
    counter = Counter()
    for i, sent in enumerate(sentences):
        counter.update(sent.split())
    
    words = [word for word, cnt in counter.items() if cnt >= threshold]
    
    vocab = Vocab()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')
    
    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab

In [53]:
vocab = build_vocab(X_train)

In [54]:
len(vocab)

1607

In [55]:
vocab('<start>')

1

In [56]:
class HinglishDataset(Dataset):
    def __init__(self, inputs, labels):
        self.sentences = inputs
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        sentiment = self.labels[idx]
        
        tokens = []
        tokens.append(vocab('<start>'))
        for token in sentence.split():
            tokens.append(vocab(token))
        tokens.append(vocab('<end>'))
        return torch.LongTensor(tokens), sentiment
    
    def collate_fn(self, data):
        data.sort(key=lambda x:len(x[0]), reverse=True)
        sentences, sentiments = zip(*data)
        
        sent_lengths = [len(sent) for sent in sentences]
        inputs = torch.zeros((len(sentences), max(sent_lengths)), dtype=torch.long)
        labels = torch.zeros(len(sentences), dtype=torch.long)
        
        for idx, sent in enumerate(sentences):
            end = sent_lengths[idx]
            inputs[idx, :end] = sent[:end]
            labels[idx] = sentiments[idx]
            
        return inputs, sent_lengths, labels
    
    def create_dataloader(self, batch_size, shuffle=False, drop_last=False):
        return DataLoader(dataset=self, batch_size=batch_size, collate_fn=self.collate_fn, 
                         shuffle=shuffle, drop_last=drop_last)

In [57]:
train_dataset = HinglishDataset(X_train, y_train)
val_dataset = HinglishDataset(X_val, y_val)
test_dataset = HinglishDataset(X_test, y_test)

In [58]:
train_dataset[0]

(tensor([ 1,  3,  4,  3,  5,  6,  3,  7,  8,  9, 10,  6,  3, 10,  6,  3, 10, 11,
          6,  3,  2]),
 0)

In [59]:
batch_size = 64
train_dataloader = train_dataset.create_dataloader(batch_size)
val_dataloader = val_dataset.create_dataloader(batch_size)
test_dataloader = test_dataset.create_dataloader(batch_size)

In [61]:
train_sample = next(iter(train_dataloader))
train_sample[0].shape, len(train_sample[1]), train_sample[2].shape

(torch.Size([64, 32]), 64, torch.Size([64]))

In [62]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, num_layers, dropout=0.4):
        super().__init__()

        self.hid_dim = hid_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(
            emb_dim,
            hid_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout
        )

        self.fc = nn.Linear(hid_dim, 50)
        self.out = nn.Linear(50, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, input_lengths):
        # inputs => [batch_size, seq_len]
        # input_lengths => [batch_size]

        embedded = self.dropout(self.embedding(inputs))

        packed_input = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, batch_first=True)
        _, hidden = self.rnn(packed_input)

        hidden = hidden.view(self.num_layers, 2, -1, self.hid_dim)
        final_forward_hidden = hidden[-1, -2, :, :]
        final_backward_hidden = hidden[-1, -1, :, :]
        # final_*_hidden => [batch_size, hidden_dim]

        combined = final_forward_hidden + final_backward_hidden
        combined = self.dropout(combined)
        # combined => [batch_size, hidden_dim]

        intermediate = F.relu(self.fc(combined))
        intermediate = self.dropout(intermediate)
        logits = self.out(intermediate)
        # logits => [batch_size, output_dim]

        return logits

In [63]:
input_dim = len(vocab)
output_dim = 3
emb_dim = 200
hid_dim = 100
num_layers = 2
NUM_EPOCHS = 20
model_path = "model/gru_model.pt"

In [65]:
model = GRUModel(input_dim, emb_dim, hid_dim, output_dim, num_layers)

In [66]:
model.to(device)

GRUModel(
  (embedding): Embedding(1607, 200)
  (rnn): GRU(200, 100, num_layers=2, batch_first=True, dropout=0.4, bidirectional=True)
  (fc): Linear(in_features=100, out_features=50, bias=True)
  (out): Linear(in_features=50, out_features=3, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [67]:
lr = 1e-4
min_lr = 3e-5
lr_decay=0.5
lr_patience=2

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', lr_decay, lr_patience, verbose=True, min_lr=min_lr)

In [72]:
def train(iterator, clip=2.0):
    epoch_loss = 0
    model.train()
    for i, batch in enumerate(iterator):
        sentences = batch[0].to(device)
        sentence_lengths = batch[1]
        targets = batch[2].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(sentences, sentence_lengths)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()
        
        if i % 2000 == 1999:
            wandb.log({"train_loss": epoch_loss/2000})
    return epoch_loss / len(iterator)

In [69]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

In [79]:
def evaluate(iterator):
    epoch_loss = 0
    model.eval()
    epoch_acc = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            sentences = batch[0].to(device)
            sentence_lengths = batch[1]
            targets = batch[2].to(device)
            logits = model(sentences, sentence_lengths)
            # logits => [batch_size, num_labels]

            loss = criterion(logits, targets)
            acc = categorical_accuracy(logits, targets)
            epoch_acc += acc.item()
            epoch_loss += loss.item()
            
            if i % 2000 == 1999:
                wandb.log({"val_loss": epoch_loss/2000})
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [71]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = elapsed_time - (elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

In [76]:
config = wandb.config
config.learning_rate = lr
config.learning_rate_decay = lr_decay
config.batch_size = batch_size

In [80]:
with wandb.init(project="Sentiment Analysis-Hinglish", config=config) as run:
    wandb.watch(model, criterion, log="all")
    best_valid_loss = float('inf')
    for epoch in range(NUM_EPOCHS):
        start_time = time.time()
        train_loss = train(train_dataloader)
        val_loss, val_acc = evaluate(val_dataloader)
        end_time = time.time()
        scheduler.step(val_loss)

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f"Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s")
        print(f"\tTrain Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Val Acc: {val_acc:.3f}")

        if val_loss < best_valid_loss:
            best_valid_loss = val_loss
            torch.save(model.state_dict(), model_path)
        model_artifact = wandb.Artifact("GRUModel", type="model")
        model_artifact.add_file(model_path)
        run.log_artifact(model_artifact)

Epoch: 01 | Time: 0m 3.61s
	Train Loss: 1.086 | Val Loss: 1.060 | Val Acc: 0.407
Epoch: 02 | Time: 0m 3.44s
	Train Loss: 1.063 | Val Loss: 0.998 | Val Acc: 0.490
Epoch: 03 | Time: 0m 3.47s
	Train Loss: 1.016 | Val Loss: 0.968 | Val Acc: 0.490
Epoch: 04 | Time: 0m 3.56s
	Train Loss: 0.989 | Val Loss: 0.960 | Val Acc: 0.491
Epoch: 05 | Time: 0m 3.59s
	Train Loss: 0.975 | Val Loss: 0.949 | Val Acc: 0.502
Epoch: 06 | Time: 0m 3.53s
	Train Loss: 0.961 | Val Loss: 0.941 | Val Acc: 0.509
Epoch: 07 | Time: 0m 3.35s
	Train Loss: 0.950 | Val Loss: 0.935 | Val Acc: 0.518
Epoch: 08 | Time: 0m 3.65s
	Train Loss: 0.945 | Val Loss: 0.931 | Val Acc: 0.524
Epoch: 09 | Time: 0m 3.39s
	Train Loss: 0.934 | Val Loss: 0.938 | Val Acc: 0.524
Epoch: 10 | Time: 0m 3.48s
	Train Loss: 0.931 | Val Loss: 0.921 | Val Acc: 0.540
Epoch: 11 | Time: 0m 3.56s
	Train Loss: 0.925 | Val Loss: 0.923 | Val Acc: 0.538
Epoch: 12 | Time: 0m 3.39s
	Train Loss: 0.917 | Val Loss: 0.924 | Val Acc: 0.543
Epoch: 13 | Time: 0m 3.52s
	

In [87]:
def evaluate_metrics(model, dataloader):
    model.eval()
    predictions = []
    targets = []
    
    with torch.no_grad():
        for batch in dataloader:
            sentences = batch[0].to(device)
            sentence_lengths = batch[1]
            targets_ = batch[2].to(device)
            predictions_ = model(sentences, sentence_lengths)
            
            outputs = predictions_.max(dim=1)[1]
            targets.extend(targets_.detach().cpu().numpy().tolist())
            predictions.extend(outputs.detach().cpu().numpy().tolist())
            
    return metrics.classification_report(targets, predictions)

In [89]:
print(evaluate_metrics(model, test_dataloader))

              precision    recall  f1-score   support

           0       0.68      0.59      0.63       982
           1       0.51      0.78      0.61       890
           2       0.49      0.34      0.40      1128

    accuracy                           0.55      3000
   macro avg       0.56      0.57      0.55      3000
weighted avg       0.56      0.55      0.54      3000

